In [22]:
# Ignore SQLITE warnings related to Decimal numbers in the Chinook database
import warnings
warnings.filterwarnings('ignore')

In [52]:
# Import Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

import numpy as np
import pandas as pd
import datetime as dt

%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

from flask import Flask, jsonify

In [53]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///../Homework/Instructions/Resources/hawaii.sqlite")

In [54]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [55]:
# Save references to each table
Measurement = Base.classes.measurement
Measurement

Station = Base.classes.station
Station

sqlalchemy.ext.automap.station

In [56]:
# Create our session (link) from Python to the DB
session = Session(engine)
session

In [58]:
# re-run your query from Part 1 of the homework:

# Most recent date

descending = session.query(Measurement.date).order_by((Measurement.date).desc())
recent_date = descending.first()

# 1 year ago from most recent date

year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Creating the query DF

precipitation = session.query(Measurement).\
    filter(Measurement.date >= year_ago).all()

rain = []
for x in precipitation:
    rain.append(x.prcp)
rain

date = []
for y in precipitation:
    date.append(y.date)
date
    
zipped = zip(date, rain)
zipped_list = list(zipped)

# setting df index

df = pd.DataFrame(zipped_list, columns =['Date', "Precipitation"])
date_index = df.set_index("Date")

# sorting the df

sorted_df = date_index.sort_values (by = 'Date')


In [59]:
# set dataframe to dictionary

dictionary_df = sorted_df.to_dict()

In [60]:
station_list = session.query(Station.station).\
    group_by(Station.station).\
    all()
station_list

[('USC00511918',),
 ('USC00513117',),
 ('USC00514830',),
 ('USC00516128',),
 ('USC00517948',),
 ('USC00518838',),
 ('USC00519281',),
 ('USC00519397',),
 ('USC00519523',)]

In [64]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

lowest_temp = (session.query(Measurement.station, func.min(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').\
    group_by(Measurement.station).\
    order_by(func.min(Measurement.station).desc()).\
    all())
#print(lowest_temp)
print(f"Station ID & Lowest temperature: {lowest_temp}")

highest_temp = (session.query(Measurement.station, func.max(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').\
    group_by(Measurement.station).\
    order_by(func.max(Measurement.station).desc()).\
    all())
#print(highest_temp)
print(f"Station ID & Highest temperature: {highest_temp}")

average_temp = (session.query(Measurement.station, func.avg(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').\
    group_by(Measurement.station).\
    order_by(func.avg(Measurement.station).desc()).\
    all())
#print(average_temp)
print(f"Station ID & Average temperature: {average_temp}")

Station ID & Lowest temperature: [('USC00519281', 54.0)]
Station ID & Highest temperature: [('USC00519281', 85.0)]
Station ID & Average temperature: [('USC00519281', 71.66378066378067)]


In [61]:
# create the connection to Flask
app = Flask(__name__)

In [62]:
# Homepage and routes
@app.route("/")
def home():
    return (
        f"Welcome to the Climate Analysis and Exploration homepage!<br/>"
        f"<br/>" 
        f"Available Routes:<br/>"
        f"<br/>" 
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end><br/>"
    )

In [63]:
@app.route("/api/v1.0/precipitation")
def precipitation1():
    return jsonify(dictionary_df)

In [66]:
@app.route("/api/v1.0/stations")
def station():
    return jsonify(station_list)

In [ ]:
@app.route("/api/v1.0/tobs")
def tobs():
    return jsonify(
        lowest_temp,
        highest_temp,
        average_temp
    )